In [3]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [4]:
%cd /content/drive/MyDrive/Pesto_assignment

/content/drive/MyDrive/Pesto_assignment


In [5]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [11]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

device_map = {"": 0}

# Fine-tuned model name
finetuned_model = "Llama-2-7b-customer-agent-finetune-5_epoch"

# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map = device_map
)
model = PeftModel.from_pretrained(base_model, finetuned_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
# prompt: build an interface here in this notebook, where user can input queries and the above loaded model will act as chat agent and return the responses

def generate_response(input_text):
    # Tokenize the input text
    input_ids = tokenizer.encode(input_text + tokenizer.eos_token, return_tensors="pt").to(model.device)

    # Generate the response
    output = model.generate(input_ids=input_ids, max_new_tokens=120)

    # Decode the output and return it
    return tokenizer.decode(output[0], skip_special_tokens=True)


Enter your query: I do not like the product. It's not exactly the same as it was mentioned in the website
Chat Agent: Aw, it sounds like you're not happy with your purchase. Sorry to hear that the product didn't meet your expectations. Can you please provide more details about the issue you're experiencing? We'll do our best to help you resolve the problem.
Enter your query: exit


In [ ]:
def calculate_perplexity(model, tokenizer, texts, max_length=2048):
    stride = 512
    lls = []

    for text in texts:
        encodings = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=max_length)
        input_ids = encodings.input_ids.to(model.device) # Move input_ids to GPU
        target_ids = input_ids.clone()

        for i in range(0, input_ids.size(1), stride):
            begin_loc = max(i + stride - max_length, 0)
            end_loc = min(i + stride, input_ids.size(1))
            trg_len = end_loc - i
            chunk_input_ids = input_ids[:, begin_loc:end_loc]
            chunk_target_ids = target_ids[:, begin_loc:end_loc].clone()
            chunk_target_ids[:, :-trg_len] = -100

            with torch.no_grad():
                outputs = model(chunk_input_ids, labels=chunk_target_ids) # Now both model and inputs are on the same device
                log_likelihood = outputs.loss * trg_len

            lls.append(log_likelihood)

    ppl = torch.exp(torch.stack(lls).sum() / end_loc)
    return ppl.item()

# Sample validation texts
validation_texts = [
    "We apologize for the inconvenience. Can you please provide your order number so we can investigate and resolve the issue?",
    "Certainly. Can you please provide your order number and the details of the item you'd like to change?",
]


perplexity = calculate_perplexity(model, tokenizer, validation_texts)
print(f'Perplexity: {perplexity}')